# Day1 — Data Summary & Basic Time-Series Figures

목표:
- processed 데이터 기준으로 기본 요약 테이블 생성(tables/Day1)
- 전체 시간축(abs_time_ms) 기준 주요 센서 시간그래프 생성(figures/Day1)
- 다음 Day2(관계 분석)를 위한 데이터 상태/패턴 확인

## 입력/출력
- 입력(processed): `data/processed/processed_v1_run_abs_features.csv`
- 출력(표): `tables/Day1/`
- 출력(그림): `figures/Day1/`

In [ ]:
from pathlib import Path
import pandas as pd

PROCESSED = Path("../../data/processed/processed_v1_run_abs_features.csv")
TABLES_DIR = Path("../../tables/Day1")
FIG_DIR = Path("../../figures/Day1")

PROCESSED.exists(), TABLES_DIR.exists(), FIG_DIR.exists()

## 1) 데이터 로드 & 기본 확인

In [ ]:
df = pd.read_csv(PROCESSED)
df.shape, df.columns[:10]

## 2) run 구조 확인
- run 개수
- run당 샘플 수(100개인지)

In [ ]:
runs = df["run_id"].nunique()
run_size_desc = df.groupby("run_id").size().describe()

runs, run_size_desc

## 3) 결측치 / 범위 sanity check

In [ ]:
missing_top = df.isna().sum().sort_values(ascending=False).head(15)

cols = ["FB_Torque", "FB_Rpm"]
if "Temp_mean" in df.columns: cols.append("Temp_mean")
if "Press_mean" in df.columns: cols.append("Press_mean")

stats = df[cols].describe().loc[["min", "max", "mean"]]

missing_top, stats

## 4) Day1 산출물 생성 (스크립트 실행)
- 표 생성: `python -m src.data_summary`
- 그림 생성: `python -m src.day1_figures_abs_time`

노트북에서는 재현성 확인을 위해 스크립트를 호출하고,
생성된 파일이 있는지만 간단히 체크한다.

In [ ]:
import subprocess
import sys

def run(cmd: str):
    print(">", cmd)
    res = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
    return res.returncode

# 윈도우/파워쉘 환경에서도 동작하도록 python 실행경로 사용
py = sys.executable

run(f"{py} -m src.data_summary")
run(f"{py} -m src.day1_figures_abs_time")

## 5) 생성 결과 확인
- tables/Day1에 CSV들이 생성되었는지 확인
- figures/Day1에 PNG들이 생성되었는지 확인

In [ ]:
tables = sorted(TABLES_DIR.glob("*.csv"))
figs = sorted(FIG_DIR.glob("*.png"))

[t.name for t in tables], [f.name for f in figs]

## 6) Day1 관찰 요약(그래프 기반)
- 전체 시간축에서 정지 → 운전 → 정지 형태의 구간이 뚜렷함
- 운전 구간에서 토크/진동 변동이 크게 나타남
- 온도(Temp_mean)는 변화 폭이 매우 작아(거의 상수) 예측/관계에서 영향이 낮을 가능성

## Day2 계획 (관계 분석)
- 관계 분석은 전체 데이터 그대로보다, 운전 구간(예: FB_Rpm > threshold) 필터링 후 수행 예정
- 산점도: RPM-토크 / 토크-진동 / 토크-압력